In [ ]:
# default_exp core

# Imports

In [ ]:
#export
from dreamai2.learner import*

################################################################################
### WARNING, path does not exist: KALDI_ROOT=/mnt/matylda5/iveselyk/Tools/kaldi-trunk
###          (please add 'export KALDI_ROOT=<your_path>' in your $HOME/.profile)
###          (or run as: KALDI_ROOT=<your_path> python <your_script>.py)
################################################################################



In [ ]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

# Functions

In [ ]:
#export
def add_to_group(img, anchor, sim_dict, group_dict):
    '''Add image to group

    Args:
        img (str)
        anchor (str)
        sim_dict (dict)
        group_dict (dict)
    '''
    sim_dict[img]['has_group'] = True
    group_dict[anchor].append(img)

def batchify_data(imgs, anchors, image_size=1024):
    '''Convert images lists to batches

    Args:
        imgs ([type]): List of images
        anchors ([type]): List of anchors
        image_size (int, optional): Defaults to 1024.
    '''
    img_batch = to_batch(paths=imgs, size=(image_size,image_size)).to(device)
    anchor_batch = to_batch(paths=anchors, size=(image_size,image_size)).to(device)

def byol_model(model_path=''):
    '''Load the trained byol model

    Args:
        model_path (str, optional): Pretrained state_dict path. Defaults to ''.
    '''
    
    model = create_body(resnet18, cut=-1, num_extra=0)
    if len(model_path) == 0  and os.path.exists(model_path):
        sd = torch.load('rnet18_byol.pt', map_location='cpu')
        sd = swap_dict_key_letters(sd, 'conv1.weight', '0.model.0.weight', strict=True)
        keys = ['bn1.weight', 'bn1.bias', 'bn1.running_mean', 'bn1.running_var', 'bn1.num_batches_tracked',]
        for k in keys:
            sd = swap_dict_key_letters(sd, k, f'0.model.1.{k.split(".")[-1]}', strict=True)
        for i in range(1,5):
            sd = swap_dict_key_letters(sd, f'layer{i}', f'0.model.{i+3}')
        model.load_state_dict(sd, strict=False)
    return model

def create_groups(imgs, anchors, model_path='', image_size=1024, bs=6, thresh=0.99, device='cpu'):
    '''Create n groups where n is the number of anchor images.

    Args:
        imgs (list): List of images
        anchors (list): List of anchor images
        model_path (str, optional): Pretrained state_dict path. Defaults to ''.
        image_size (int, optional): Defaults to 1024.
        bs (int, optional): Batch size. Defaults to 6.
        thresh (float, optional): Similarity threshold. Defaults to 0.99.
        device (str, optional): Defaults to 'cpu'.
    '''
    
    if not is_str(imgs[0]):
        imgs = [str(img) for img in imgs]
        
    if not is_str(anchors[0]):
        anchors = [str( anchor) for anchor in anchors]
    
    model = byol_model(model_path=model_path)
    device = torch.device(device)
    model.eval().to(device)
    
    anchor_df = pd.DataFrame({'Anchors':anchors})    
    img_df = pd.DataFrame({'Images':imgs})
    anchor_ds = DaiDataset(data=anchor_df, tfms=instant_tfms(h=image_size, w=image_size, test_tfms=False))
    img_ds = DaiDataset(data=img_df, tfms=instant_tfms(h=image_size, w=image_size, test_tfms=False))
    anchor_dl = DataLoader(anchor_ds, batch_size=bs, shuffle=False, num_workers=4, pin_memory=True)
    img_dl = DataLoader(img_ds, batch_size=bs, shuffle=False, num_workers=4, pin_memory=True)
    
    anchor_vectors = []
    img_vectors = []
    
    with torch.no_grad():
        for anchor_batch in anchor_dl:
            anchor_vectors.append(model(anchor_batch['x'][0].to(device)).flatten(start_dim=1))
        for img_batch in img_dl:
            img_vectors.append(model(img_batch['x'][0].to(device)).flatten(start_dim=1))
    anchor_vectors = torch.cat(anchor_vectors)
    img_vectors = torch.cat(img_vectors)
    anchor_vectors = [x.repeat(len(img_vectors),1) for x in anchor_vectors]
    
    cos = nn.CosineSimilarity(dim=1, eps=1e-6)
    sims = [cos(x, img_vectors) for x in anchor_vectors]
    
    sim_dict = {img:{'has_group':False, 'best':0, 'best_anchor':''} for img in imgs}
    group_dict = {anchor:[] for anchor in anchors}
    for i,anchor in enumerate(anchors):
        for j,img in enumerate(imgs):
            sim = sims[i][j]
            if sim >= sim_dict[img]['best']:
                sim_dict[img]['best'] = sim.detach().item()
                sim_dict[img]['best_anchor'] = anchor
                if sim >= thresh:
                    add_to_group(img, anchor, sim_dict, group_dict)
            if i == len(anchors)-1 and not sim_dict[img]['has_group']:
                add_to_group(img, sim_dict[img]['best_anchor'], sim_dict, group_dict)
    return group_dict


In [ ]:
# #cuda

# data_name = 'final_demo'
# data_path = Path(f'/media/hamza/data2/{data_name}/')
# images_path = data_path/'zoey_synced'

# all_imgs = sorted_paths(images_path, key=path_name, make_str=True)
# anchors = [all_imgs[i] for i in range(0,len(all_imgs),3)]
# imgs = flatten_list([all_imgs[i+1:i+3] for i in range(0,len(all_imgs),3)])

# device = default_device()

# group_dict = create_groups(imgs, anchors, model_path='rnet18_byol.pt', image_size=1024, bs=3, device=device)

# assert sorted(dict_keys(group_dict)) == sorted(anchors)

# assert sorted(dict_keys(create_groups(imgs, anchors=[Path(a) for a in anchors], model_path='', image_size=1024, bs=3, device=device))) == sorted(anchors)

# assert len(flatten_list(dict_values(group_dict))) == len(imgs)

[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warn